In [14]:
import polars as pl
import pandas as pd
import os

full_datasets_path = "App/datasets"

datasets_list = [os.path.join(full_datasets_path, item) for item in os.listdir(full_datasets_path) 
                if os.path.isdir(os.path.join(full_datasets_path, item))]
df_metrics = pd.DataFrame()

for dataset_path in datasets_list:
    experiments_folder = os.path.join(dataset_path, "runs")
    if os.path.exists(experiments_folder):
        runs_folders = [os.path.join(experiments_folder, run_folder) for run_folder in os.listdir(experiments_folder)]
        
        
        for run_folder in runs_folders:
            df_metrics_run = pd.read_csv(os.path.join(run_folder, "training_kfold(10)_metrics.csv"))
            df_metrics_run["dataset"] = dataset_path.split("/")[-1]
            df_metrics = pd.concat([df_metrics, df_metrics_run])

df_metrics = df_metrics[["dataset"] + df_metrics.columns.drop("dataset").tolist()]
df_metrics.fillna("-")

FileNotFoundError: [Errno 2] No such file or directory: 'App/datasets/dataset24_bonidia_dnarna/runs/run_1/training_kfold(10)_metrics.csv'

In [38]:
with open("antioxidant.fasta", "w") as output_handle:
    # iterate up to the penultimate element
    for row in df_data.iterrows():
        # current = seqs[i]
        row_series = row[1]

        output_handle.write(f">antioxidant\n")
        output_handle.write(f"{row_series[0]}\n")

In [15]:
pl.read_csv("App/datasets/dataset11_lv_dnarna/runs/run_1/feat_extraction/Tsallis.csv", ignore_errors=True) \
.select(pl.all().exclude("nameseq"), pl.col("nameseq"))\
.filter(~pl.col("nameseq").str.contains("nameseq")) \
.set_sorted("nameseq")

k-tsallis12.3,k-tsallis22.3,k-tsallis32.3,k-tsallis42.3,k-tsallis52.3,label,nameseq
f64,f64,f64,f64,f64,str,str
0.600994,0.728334,0.755018,0.760864,0.762194,"""negative""","""-sample"""
0.628107,0.740318,0.756621,0.759818,0.761081,"""negative""","""-sample"""
0.61398,0.732599,0.754893,0.760864,0.762194,"""negative""","""-sample"""
0.610519,0.727303,0.751481,0.760341,0.762194,"""negative""","""-sample"""
0.623592,0.735025,0.756621,0.761387,0.762194,"""negative""","""-sample"""
…,…,…,…,…,…,…
0.633963,0.736455,0.756128,0.759818,0.761081,"""positive""","""+sample"""
0.633963,0.738939,0.758592,0.760864,0.761637,"""positive""","""+sample"""
0.579598,0.710172,0.74529,0.755017,0.759272,"""positive""","""+sample"""
